# Data Exploratie 

### Phase 1: Business Understanding

Het doel van dit model is om te testen of populariteit van een nummer 
<br>
voorspelt kan worden met lineaire regressie o.b.v de audio-features 
<br>
in deze dataset. 

### Phase 2: Data Understanding 

In [1]:
# Modules importeren
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

# Dataset importeren 
df = pd.read_csv("/Users/odessa/Desktop/Applied Data Science & AI/Data Science/Code Inleiding data science/song_data.csv")

# Target variabele maken 
target = 'song_popularity'
#df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

In [ ]:
# Eerste 5 rijen van de dataframe printen
display(df.head())
# Er zijn in totaal 16 kolommen in deze dataframe

In [ ]:
# Datatypes printen en checken of er nullwaardes zijn 
df.info()

In [ ]:
# Statistiche informatie weergeven 
df.describe().T

# Opvallend: 
# Time_signature van 0 
# Loudness heeft normaliter een waarde tussen -60 en 0 dB 
# Tempo heeft een minimum van 0

# danceability minimum = 0 checken ?
# speechiness minimum = 0 checken ?

# Deze punten worden aangepakt bij fase 3 

In [ ]:
# Hoeveel waardes zitten in de dataframe? 
print(f"Totaal aantal waardes in de dataframe: {len(df)}")

# Hoeveel unieke rijen in elke kolom? 
df.nunique().sort_values(ascending=False)

### Grafieken plotten 

In [ ]:
# Matplotlib histogrammen plotten 
df.hist(figsize=(15,12), bins=30)
plt.tight_layout()
plt.show

In [ ]:
# Plot popularity distribution
plt.hist(df["song_popularity"], bins=20, color="skyblue", edgecolor="black")
plt.title("Popularity Distribution")
plt.xlabel("Popularity (0-100)")
plt.ylabel("Number of songs")
plt.show()

### Correlaties


0.00 – 0.30	nauwelijks of geen correlatie
<br>
0.30 – 0.50	lage of zwakke correlatie
<br>
0.50 – 0.70	middelmatige correlatie
<br>
0.70 – 0.90	hoge of sterke correlatie
<br>
0.90 – 1.00	zeer hoge of zeer sterke correlatie

In [ ]:
df_num = df.select_dtypes(include=['number']).copy()
corr_matrix = df_num.corr()
print(corr_matrix['song_popularity'].sort_values(ascending=True))

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title("Correlatie heatmap")
plt.show()

# Loudness en energy hebben een sterke positieve correlatie: 0.77
# energy en acousticness hebben een redelijk sterke negatieve correlatie: -0.68
# loudness en acousticness hebben een negatieve correlatie: -0.57 
# loudness en instrumentallness hebben een negatieve correlatie: -0.4

In [ ]:
attributes = ["energy", "loudness", "acousticness",
"instrumentalness"]
scatter_matrix(df[attributes], figsize=(12, 8))
plt.show()

### Phase 3: Data Preparation

In [ ]:
# time_signature gaat van 3 tot 7 volgens spotify api
df["time_signature"].value_counts()
# waardes van 0 en 1 kan niet 

In [ ]:
# Indexen van time_signature == 0 
time_sig_is_zero = df[df['time_signature'] == 0]
print(time_sig_is_zero['time_signature'])

In [ ]:
# Indexen van time_signature == 1
time_sig_is_one = df[df['time_signature'] == 1]
print(time_sig_is_one['time_signature'])

In [ ]:
# Dataframe maken van time_signatures 0 en 1 
df_time_signature_0_1 = pd.concat([time_sig_is_zero, time_sig_is_one])

In [ ]:
# Statistische berekeningen zien van waardes met time signatures van 0 en 1 
display(df_time_signature_0_1.describe())

In [ ]:
display(df_time_signature_0_1.sort_values(by='song_duration_ms').head(25))
#TO-DO: index 7119 verwijderen 

In [ ]:
display(df_time_signature_0_1.sort_values(by='danceability').head(25))
# TO-DO: index 11171 verwijderen want speechiness, danceability en tempo = 0, 

In [ ]:
# Nummers met een decibelvolume boven 0. 
luidste_nummers = df.loc[df['loudness'] > 0, ['song_name', 'loudness', 'song_popularity']]
print(luidste_nummers)

# 7 waardes. 

In [ ]:
# Tempo waardes sorteren van laag naar hoog 
df = df.sort_values('tempo')

# Nummers met een tempo van 0. 
nul_tempo = df.loc[df['tempo'] == 0, ['song_name', 'tempo', 'song_popularity']]
print(f'Nummers met een tempo van 0: {nul_tempo}')

# Tempo == 0 is opgelost na verwijderen nummers 9499 en 6415

In [ ]:
# Bij song_name veel dubbele waardes 
totaal_song_name = df['song_name'].count()
unieke_song_name = df['song_name'].nunique()
dubbel_song_name = totaal_song_name - unieke_song_name

print(f"song_name heeft {dubbel_song_name} dubbele waardes")

In [ ]:
df['instrumentalness'].value_counts()